# 5일차 2교시 - Spark Skewness Problem
> 특정 킷 값이 너무 많아 일부 Reduce 작업이 지연되어 전체 작업 시간에 영향을 주는 경우를 해결합니다

### 목차
* [1. Skewness 문제의 접근](#1.-Skewness-문제의-접근)
* [2. Skewness 편중의 인지](#2.-Skewness-편중의-인지)
* [3. Skewness 해결 전략](#3.-Skewness-해결-전략)
* [4. Skewness 전략 실습](#4.-Skewness-전략-실습)

### 참고자료
  * https://itnext.io/handling-data-skew-in-apache-spark-9f56343e58e8

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark

21/08/21 13:08:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### 1. Skewness 문제의 접근
#### 1.1. Skew 를 발생시키는 데이터를 반드시 사용해야만 하는가?
> 대부분의 Skew 발생 대상 컬럼은 outer join 혹은 데이터 누락에 의한 Null 혹은 0 인 경우가 많은데 해당 데이터가 필요 없다면 제외합니다

#### 1.2. Skew 를 발생 시키는 데이터를 별도의 파이프라인을 구성할 필요가 있는가?
> Skew 발생 대상 컬럼에 대해 처리나 별도의 연산이 필요하다면, Skew 대상 데이터를 별도의 데이터프레임으로 구성하고 Union 을 통해 최종 결과를 생성하는 것이 병렬성 및 향후 유지보수 관점에도 유용할 수 있습니다

#### 1.3. 조인 대상 테이블 가운데 Broadcast 해도 좋을 만큼 충분히 작은 테이블이 존재하는가?
> 브로드캐스팅 대상 테이블을 큰 테이블로 전송하는 방식이므로 파티셔닝의 수를 변경할 수 없는 제약이 있어 큰 테이블이 사전에 잘 파티셔닝 혹은 evenly distributed 되어 있어야 더욱 좋은 효과를 발휘합니다
> 브로드캐스팅 할 수 있는 크기가 최대 8gb 로 제한되어 있어 그 이상 전송할 수 없으며, 기본 값은 10mb 입니다.
> 브로드캐스팅은 당연하게도 해당 브로드캐스팅 테이블은 드라이브 컨테이너 뿐만 아니라 드라이버의 메모리에 모두 올라올 수 있을 만큼 충분한 메모리가 필요합니다. 예를 들어 1gb 짜리 테이블이고 100개의 컨테이너가 뜬다고 가정하면 최소 총 100gb 이상의 추가적인 메모리 오버헤드가 발생합니다. 자칫 잘못하면 아래와 같은 메시지와 함께 어플리케이션이 종료됩니다.
```java
java.lang.OutOfMemoryError: Not enough memory to build and broadcast the table to all worker nodes.
```
> 브로드캐스팅은 해당 데이터 전체가 네트워크를 모두 점유하고 해당 컨테이너가 해당 테이블 전체를 복사해야 하므로 상당히 무거운 작업입니다. 그러한 부담을 감수할 만큼 충분히 가치가 있는 경우에만 적용할 수 있습니다. 
> 특히 브로드캐스팅 대상 테이블이 점점 커지는 경우라면 언젠가는 임계치를 초과하여 문제가 발생할 수 있으므로 데이터의 특성을 잘 고려해야만 합니다.

#### 1.4 Skew 대상 컬럼을 특정할 수 있으며 브로드캐스팅이 어렵다면?
> 치우친 데이터 컬럼에 대해 추가 랜덤 키(salt key)를 부여하고 조인 되는 대상 테이블에는 해당 salt key 최대 크기 만큼 뻥튀기 (explode) 하여 키 파티셔닝을 통한 병렬성을 늘릴 수 있도록 합니다 .
* saltedJoin
```python
def saltedJoin(df: DataFrame, buildDf: DataFrame, joinExpression: Column, joinType: String, salt: Int): DataFrame = {
    import org.apache.spark.sql.functions._
    val tmpDf = buildDf.withColumn(“slt_range”, array(Range(0, salt).toList.map(lit): _*))
    val tableDf = tmpDf.withColumn(“slt_ratio_s”, explode(tmpDf(“slt_range”))).drop(“slt_range”)
    val streamDf = df.withColumn(“slt_ratio”, monotonically_increasing_id % salt)
    val saltedExpr = streamDf(“slt_ratio”) === tableDf(“slt_ratio_s”) && joinExpression
    streamDf.join(tableDf, saltedExpr, joinType).drop(“slt_ratio_s”).drop(“slt_ratio”)
}
```
> 즉, 메인 테이블에 A 라는 키가 존재하고 조인 되는 테이블에도 A 가 있다면 메인 테이블에는 임의의 1~N 까지의 킷 값을 부여하고 조인되는 테이블의 레코드 수를 N배로 explode 하게 하여 동일한 값을 N배 레코드로 확장하여 조인할 수 있도록 하고, 마지막에 해당 salting key 컬럼을 제거합니다 
```python
val df = spark.read.parquet(“s3://...”)
val geoDataDf = spark.read.parquet(“s3://...”)
val userAgentDf = spark.read.parquet(“s3://...”)
val ownerMetadataDf = spark.read.parquet(“s3://...”)
df
 .saltedJoin(geoDataDf, exprGeo, “left”, 200)
 .saltedJoin(userAgentDf, exprUserAgent, “left”, 200)
 .saltedJoin(ownerMetadataDf, exprOwnerMetadata, “left”, 200)
 .write
 .parquet(“s3://...”)
```

In [2]:
small = spark.range(1, 100)
medium = spark.range(1, 1000)
large = spark.range(1, 10000)

### 2. Skewness 편중의 인지
>  조인 연산에 있어서 파티션 단위 데이터 스큐를 해결하기 어려운 이유는 임의의 컬럼에 대해 조인 연산을 할 때에 해당 키를 기준으로 말아올렸을 때의 파티셔닝이 잘 분산되어 있다는 보장도 어렵고 사전에 예측도 어렵기 때문이다. 예를 들어 조인 조건이 A.name == B.name and A.model == B.model 일 경우 name 과 model 에 의해 생성되는 일치하는 파티셔닝 그룹이 어떤 그룹에 많은 데이터가 모일 지 예측하기 어렵고, 시간이 지남에 따라 혹은 특정 시기에 변화할 수도 있기 때문이다.

* 파티션 단위 데이터 편중 현상 예측이 어려운 이유

### 3. Skewness 해결 전략

#### 3.1 최대한 조인 대상 범위를 줄입니다
> 전체 대상으로 조인을 하는 것 보다 최대한 필요한 컬럼만 지정하고, 구체적인 필터를 통해 조인 전에 데이터를 줄입니다

#### 3.2 메모리에 올릴 수 있는 상대적으로 작은 테이블인지 확인
> 조인대상 테이블 중에 충분히 작은 10~100mb 미만의 경우 broadcast 힌트를 통해 메모리에 올려 조인합니다

#### 3.3 조인 키에 추가할 만한 데이터가 있는지 확인
> 우리가 원하는 것은 가장 큰 파티션인 Ford Fiesta 을 작은 파티션들로 쪼개어 병렬처리가 가능하게 하기 위함
0.1 리터 차이가 나는 필터를 적용하면 직관적이지만, 파티션이 "model", "make" 에 의해 결정나기 때문에 스큐현상을 피할 수 없습니다
이를 회피하기 위해 exact matching 이 가능하도록 -1, 0, +1 의 3가지 경우를 explode 를 통해 생성해내어 조인을 수행합니다
이 경우는 engine_size 가 0.1 단위로만 차이가 난다는 가정을 해야만 하지만, 비지니스 로직이 명확하다면 가장 좋은 성능을 냅니다

#### 3.4 조인 대상 테이블에 랜덤한 수를 추가하여 병렬성을 추가하는 방법
> 상대적으로 작은 테이블에 병렬수 만큼의 시퀀스를 추가해서 뻥튀기 합니다
상대적으로 큰 테이블에는 파티션 별로 일정하게 증가하는 시퀀스를 추가하여 조인할 수 있도록 구성합니다

#### 3.5 병렬 수준을 높이거나, 스큐가 발생하는 컬럼에 대해서만 병렬성을 추가하는 방법
> 충분히 많은 타스크 수를 늘려서 병렬성을 높이는 방법
정해진 컬럼에 대해서만 병렬성을 늘리고, 다른 컬럼에 대해서는 일정한 숫자를 추가합니다

In [2]:
s1 = spark.read.parquet("source/s1")
s2 = spark.read.parquet("source/s2")
s1.printSchema()
c1 = s1.count()
c2 = s2.count()
print(c1, c2)

root
 |-- registration: string (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- engine_size: decimal(38,18) (nullable = true)

10000 100000


In [4]:
s1.show(10)

+------------+-------------+-------+--------------------+
|registration|         make|  model|         engine_size|
+------------+-------------+-------+--------------------+
|     CPbYgbw|         FORD| FIESTA|1.300000000000000000|
|     Q8GO2EU|         FORD| FIESTA|1.000000000000000000|
|     qTQ7HxY|         FORD| FIESTA|1.200000000000000000|
|     3FpMCC8|     VAUXHALL|  CORSA|1.500000000000000000|
|     cirRCzK|       NISSAN|QASHQAI|1.300000000000000000|
|     mK5LtWT|         FIAT|    500|1.100000000000000000|
|     Cjxu9je|MERCEDED_BENZ|E CLASS|1.600000000000000000|
|     j359V7w|       NISSAN|QASHQAI|1.100000000000000000|
|     oaCQJN0|         FORD| FIESTA|1.300000000000000000|
|     eGqJCKX|MERCEDED_BENZ|E CLASS|1.500000000000000000|
+------------+-------------+-------+--------------------+
only showing top 10 rows



In [5]:
s2.show(10)

+-------------+-------+--------------------+----------+
|         make|  model|         engine_size|sale_price|
+-------------+-------+--------------------+----------+
|         FIAT|    500|1.100000000000000000|    1610.0|
|          KIA|    RIO|1.800000000000000000|    1934.0|
|       SUZUKI|  SWIFT|1.400000000000000000|     946.0|
|       SUZUKI|  SWIFT|1.200000000000000000|    4799.0|
|         FIAT|    500|1.400000000000000000|    5213.0|
|       SUZUKI|  SWIFT|1.600000000000000000|    2529.0|
|       NISSAN|QASHQAI|1.100000000000000000|    2120.0|
|          KIA|    RIO|1.200000000000000000|    2122.0|
|         FORD| FIESTA|1.300000000000000000|     862.0|
|MERCEDED_BENZ|E CLASS|1.100000000000000000|    2456.0|
+-------------+-------+--------------------+----------+
only showing top 10 rows



In [6]:
from pyspark.sql.functions import *

cond = [s1.make == s2.make, s1.model == s2.model]
s1.join(s2, cond).filter(s2.engine_size - s1.engine_size <= 0.1).groupBy("registration").agg(avg("sale_price")).explain()
# .filter(abs(s2("engine_size") - s1("engine_size")) <= "0.1")
# .groupBy("registration").agg(avg("sale_price").as("average_price"))

== Physical Plan ==
*(3) HashAggregate(keys=[registration#6], functions=[avg(sale_price#17)])
+- Exchange hashpartitioning(registration#6, 5), ENSURE_REQUIREMENTS, [id=#144]
   +- *(2) HashAggregate(keys=[registration#6], functions=[partial_avg(sale_price#17)])
      +- *(2) Project [registration#6, sale_price#17]
         +- *(2) BroadcastHashJoin [make#7, model#8], [make#14, model#15], Inner, BuildLeft, (cast(CheckOverflow((promote_precision(cast(engine_size#16 as decimal(38,17))) - promote_precision(cast(engine_size#9 as decimal(38,17)))), DecimalType(38,17), true) as double) <= 0.1), false
            :- BroadcastExchange HashedRelationBroadcastMode(List(input[1, string, false], input[2, string, false]),false), [id=#135]
            :  +- *(1) Filter (isnotnull(make#7) AND isnotnull(model#8))
            :     +- *(1) ColumnarToRow
            :        +- FileScan parquet [registration#6,make#7,model#8,engine_size#9] Batched: true, DataFilters: [isnotnull(make#7), isnotnull(model#8

In [7]:
s1.groupBy("make", "model").count().sort(col("count").desc()).show(3)

+--------+------+-----+
|    make| model|count|
+--------+------+-----+
|    FORD|FIESTA| 5720|
|    FIAT|   500|  574|
|VAUXHALL| CORSA|  556|
+--------+------+-----+
only showing top 3 rows



In [8]:
s2.groupBy("make", "model").count().sort(col("count").desc()).show(3)

+-------+------+-----+
|   make| model|count|
+-------+------+-----+
|   FORD|FIESTA|55584|
|   FIAT|   500| 5695|
|HYUNDAI|   I20| 5646|
+-------+------+-----+
only showing top 3 rows



### 4. Skewness 전략 실습

#### 4.1. 브로드캐스팅을 통한 조인연산
> 데이터의 크기가 충분히 작아서 브로드캐스팅 되지만, 명시적으로 크기를 지정합니다, http://localhost:4040/stages/ 페이지에서 전체적으로 소요되는 시간을 확인합니다.

In [9]:
%%time
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10*1024*1024)  # 10mb
res = s1.join(s2, cond).filter(s2.engine_size - s1.engine_size <= 0.1).groupBy("registration").agg(avg("sale_price"))
res.explain()
res.show(10)

== Physical Plan ==
*(3) HashAggregate(keys=[registration#6], functions=[avg(sale_price#17)])
+- Exchange hashpartitioning(registration#6, 5), ENSURE_REQUIREMENTS, [id=#272]
   +- *(2) HashAggregate(keys=[registration#6], functions=[partial_avg(sale_price#17)])
      +- *(2) Project [registration#6, sale_price#17]
         +- *(2) BroadcastHashJoin [make#7, model#8], [make#14, model#15], Inner, BuildLeft, (cast(CheckOverflow((promote_precision(cast(engine_size#16 as decimal(38,17))) - promote_precision(cast(engine_size#9 as decimal(38,17)))), DecimalType(38,17), true) as double) <= 0.1), false
            :- BroadcastExchange HashedRelationBroadcastMode(List(input[1, string, false], input[2, string, false]),false), [id=#263]
            :  +- *(1) Filter (isnotnull(make#7) AND isnotnull(model#8))
            :     +- *(1) ColumnarToRow
            :        +- FileScan parquet [registration#6,make#7,model#8,engine_size#9] Batched: true, DataFilters: [isnotnull(make#7), isnotnull(model#8

+------------+------------------+
|registration|   avg(sale_price)|
+------------+------------------+
|     iXXrlpz|2977.1369648060972|
|     3LIElNn|2977.1369648060972|
|     HZJtAcH| 3022.568145376804|
|     cYzmEAU|2977.1369648060972|
|     n22OxDp| 2987.987181738367|
|     PTwNpKh| 2987.987181738367|
|     O7ydunD| 2985.096955128205|
|     qo0NBCn| 2987.987181738367|
|     Fwsi8dg| 2986.729458388375|
|     wiFO8E9|2978.1062155782847|
+------------+------------------+
only showing top 10 rows

CPU times: user 18.5 ms, sys: 8.4 ms, total: 26.9 ms
Wall time: 44.2 s


#### 4.2. 셔플링을 통한 조인
> 브로드캐스팅 되지 않도록 임계치 값을 -1로 지정하여 브로드캐스팅 되지 않도록 설정하고, http://localhost:4040/stages/ 페이지에서 가장 시간이 오래걸린 작업을 확인합니다.

In [10]:
%%time
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
res = s1.join(s2, cond).filter(s2.engine_size - s1.engine_size <= 0.1).groupBy("registration").agg(avg("sale_price"))
res.explain()
res.show(10)

== Physical Plan ==
*(6) HashAggregate(keys=[registration#6], functions=[avg(sale_price#17)])
+- Exchange hashpartitioning(registration#6, 5), ENSURE_REQUIREMENTS, [id=#431]
   +- *(5) HashAggregate(keys=[registration#6], functions=[partial_avg(sale_price#17)])
      +- *(5) Project [registration#6, sale_price#17]
         +- *(5) SortMergeJoin [make#7, model#8], [make#14, model#15], Inner, (cast(CheckOverflow((promote_precision(cast(engine_size#16 as decimal(38,17))) - promote_precision(cast(engine_size#9 as decimal(38,17)))), DecimalType(38,17), true) as double) <= 0.1)
            :- *(2) Sort [make#7 ASC NULLS FIRST, model#8 ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(make#7, model#8, 5), ENSURE_REQUIREMENTS, [id=#413]
            :     +- *(1) Filter (isnotnull(make#7) AND isnotnull(model#8))
            :        +- *(1) ColumnarToRow
            :           +- FileScan parquet [registration#6,make#7,model#8,engine_size#9] Batched: true, DataFilters: [is

+------------+------------------+
|registration|   avg(sale_price)|
+------------+------------------+
|     VAXwDvW|2949.7984683595323|
|     oOwbLbj| 2940.165147453083|
|     atj0cou| 2957.128184713376|
|     UPDkPxK| 2957.128184713376|
|     10MC6Fc| 2972.469609724888|
|     JgYTNw4|3000.4284736481864|
|     STRW2oL|2949.7984683595323|
|     EQ1DjHV| 2957.128184713376|
|     J1QgrWA| 2957.128184713376|
|     qFiJQxM| 2972.469609724888|
+------------+------------------+
only showing top 10 rows

CPU times: user 56.9 ms, sys: 29.2 ms, total: 86.1 ms
Wall time: 1min 32s


![spark2](images/spark-2-tasks.png)
![spark2](images/spark-2-skewness.png)

#### 4.3 솔팅 기법을 통한 조인

In [11]:
numbers = spark.range(1, 5)
numbers.withColumn("new", explode(array([lit(x) for x in range(0,5)]))).show()

+---+---+
| id|new|
+---+---+
|  1|  0|
|  1|  1|
|  1|  2|
|  1|  3|
|  1|  4|
|  2|  0|
|  2|  1|
|  2|  2|
|  2|  3|
|  2|  4|
|  3|  0|
|  3|  1|
|  3|  2|
|  3|  3|
|  3|  4|
|  4|  0|
|  4|  1|
|  4|  2|
|  4|  3|
|  4|  4|
+---+---+



In [11]:
%%time
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
magic_number = 3
t1 = s1.withColumn("skew_key", explode(lit(array([lit(x) for x in range(0, magic_number)]))))
t2 = s2.withColumn("skew_key", monotonically_increasing_id() % magic_number)
cond = [t1.make == t2.make, t1.model == t2.model, t1.skew_key == t2.skew_key]
res = t1.join(t2, cond).filter(s2.engine_size - s1.engine_size <= 0.1).groupBy("registration").agg(avg("sale_price"))
res.explain()
res.show(10)

== Physical Plan ==
*(6) HashAggregate(keys=[registration#0], functions=[avg(sale_price#11)])
+- Exchange hashpartitioning(registration#0, 5), ENSURE_REQUIREMENTS, [id=#894]
   +- *(5) HashAggregate(keys=[registration#0], functions=[partial_avg(sale_price#11)])
      +- *(5) Project [registration#0, sale_price#11]
         +- *(5) SortMergeJoin [make#1, model#2, cast(skew_key#170 as bigint)], [make#8, model#9, skew_key#176L], Inner, (cast(CheckOverflow((promote_precision(cast(engine_size#10 as decimal(38,17))) - promote_precision(cast(engine_size#3 as decimal(38,17)))), DecimalType(38,17), true) as double) <= 0.1)
            :- *(2) Sort [make#1 ASC NULLS FIRST, model#2 ASC NULLS FIRST, cast(skew_key#170 as bigint) ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(make#1, model#2, cast(skew_key#170 as bigint), 5), ENSURE_REQUIREMENTS, [id=#875]
            :     +- Generate explode([0,1,2]), [registration#0, make#1, model#2, engine_size#3], false, [skew_key#170]
 

+------------+------------------+
|registration|   avg(sale_price)|
+------------+------------------+
|     mK5LtWT| 3022.568145376804|
|     W3sj1FD| 2986.729458388375|
|     f0ohMxn|2977.1369648060972|
|     Cn2bGX8| 2985.096955128205|
|     I7cJySY| 3022.568145376804|
|     uq0hBYu|2977.1369648060972|
|     mOe4qzJ|2983.3983024206223|
|     71LjoLT| 2985.096955128205|
|     tKWT46N|2977.1369648060972|
|     fRg72LG| 2986.729458388375|
+------------+------------------+
only showing top 10 rows

CPU times: user 21.6 ms, sys: 8.55 ms, total: 30.2 ms
Wall time: 41.4 s


In [12]:
%%time
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
magic_number = 4
t1 = s1.withColumn("skew_key", explode(lit(array([lit(x) for x in range(0, magic_number)]))))
t2 = s2.withColumn("skew_key", monotonically_increasing_id() % magic_number)
cond = [t1.make == t2.make, t1.model == t2.model, t1.skew_key == t2.skew_key]
res = t1.join(t2, cond).filter(s2.engine_size - s1.engine_size <= 0.1).groupBy("registration").agg(avg("sale_price"))
res.explain()
res.show(10)

== Physical Plan ==
*(6) HashAggregate(keys=[registration#0], functions=[avg(sale_price#11)])
+- Exchange hashpartitioning(registration#0, 5), ENSURE_REQUIREMENTS, [id=#1091]
   +- *(5) HashAggregate(keys=[registration#0], functions=[partial_avg(sale_price#11)])
      +- *(5) Project [registration#0, sale_price#11]
         +- *(5) SortMergeJoin [make#1, model#2, cast(skew_key#230 as bigint)], [make#8, model#9, skew_key#236L], Inner, (cast(CheckOverflow((promote_precision(cast(engine_size#10 as decimal(38,17))) - promote_precision(cast(engine_size#3 as decimal(38,17)))), DecimalType(38,17), true) as double) <= 0.1)
            :- *(2) Sort [make#1 ASC NULLS FIRST, model#2 ASC NULLS FIRST, cast(skew_key#230 as bigint) ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(make#1, model#2, cast(skew_key#230 as bigint), 5), ENSURE_REQUIREMENTS, [id=#1072]
            :     +- Generate explode([0,1,2,3]), [registration#0, make#1, model#2, engine_size#3], false, [skew_key#23

+------------+------------------+
|registration|   avg(sale_price)|
+------------+------------------+
|     mK5LtWT| 3022.568145376804|
|     W3sj1FD| 2986.729458388375|
|     f0ohMxn|2977.1369648060972|
|     Cn2bGX8| 2985.096955128205|
|     I7cJySY| 3022.568145376804|
|     uq0hBYu|2977.1369648060972|
|     mOe4qzJ|2983.3983024206223|
|     71LjoLT| 2985.096955128205|
|     tKWT46N|2977.1369648060972|
|     fRg72LG| 2986.729458388375|
+------------+------------------+
only showing top 10 rows

CPU times: user 24.3 ms, sys: 8.36 ms, total: 32.6 ms
Wall time: 51.5 s


In [6]:
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewJoin.skewedPartitionFactor", 1)
spark.conf.set("spark.sql.adaptive.skewJoin.skewedPartitionThresholdInBytes", 10*1024)
spark.conf.set("spark.sql.adaptive.advisoryPartitionSizeInBytes", 10*1024)

In [7]:
%%time
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
cond = [s1.make == s2.make, s1.model == s2.model]
res = s1.join(s2, cond).filter(s2.engine_size - s1.engine_size <= 0.1).groupBy("registration").agg(avg("sale_price"))
res.explain()
res.show(10)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[registration#0], functions=[avg(sale_price#11)])
   +- Exchange hashpartitioning(registration#0, 5), ENSURE_REQUIREMENTS, [id=#382]
      +- HashAggregate(keys=[registration#0], functions=[partial_avg(sale_price#11)])
         +- Project [registration#0, sale_price#11]
            +- SortMergeJoin [make#1, model#2], [make#8, model#9], Inner, (cast(CheckOverflow((promote_precision(cast(engine_size#10 as decimal(38,17))) - promote_precision(cast(engine_size#3 as decimal(38,17)))), DecimalType(38,17), true) as double) <= 0.1)
               :- Sort [make#1 ASC NULLS FIRST, model#2 ASC NULLS FIRST], false, 0
               :  +- Exchange hashpartitioning(make#1, model#2, 5), ENSURE_REQUIREMENTS, [id=#374]
               :     +- Filter (isnotnull(make#1) AND isnotnull(model#2))
               :        +- FileScan parquet [registration#0,make#1,model#2,engine_size#3] Batched: true, DataFilters: [isnotnull(make#1)

+------------+------------------+
|registration|   avg(sale_price)|
+------------+------------------+
|     VAXwDvW|2949.7984683595323|
|     oOwbLbj| 2940.165147453083|
|     atj0cou| 2957.128184713376|
|     UPDkPxK| 2957.128184713376|
|     10MC6Fc| 2972.469609724888|
|     JgYTNw4|3000.4284736481864|
|     STRW2oL|2949.7984683595323|
|     EQ1DjHV| 2957.128184713376|
|     J1QgrWA| 2957.128184713376|
|     qFiJQxM| 2972.469609724888|
+------------+------------------+
only showing top 10 rows

CPU times: user 31.3 ms, sys: 13.5 ms, total: 44.8 ms
Wall time: 41.6 s


![spark3](images/spark-3-tasks.png)
![spark3](images/spark-3-skewness.png)


In [9]:
spark.conf.set("spark.sql.adaptive.enabled", "false")

In [10]:
%%time
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
cond = [s1.make == s2.make, s1.model == s2.model]
res = s1.join(s2, cond).filter(s2.engine_size - s1.engine_size <= 0.1).groupBy("registration").agg(avg("sale_price"))
res.explain()
res.show(10)

== Physical Plan ==
*(6) HashAggregate(keys=[registration#0], functions=[avg(sale_price#11)])
+- Exchange hashpartitioning(registration#0, 5), ENSURE_REQUIREMENTS, [id=#708]
   +- *(5) HashAggregate(keys=[registration#0], functions=[partial_avg(sale_price#11)])
      +- *(5) Project [registration#0, sale_price#11]
         +- *(5) SortMergeJoin [make#1, model#2], [make#8, model#9], Inner, (cast(CheckOverflow((promote_precision(cast(engine_size#10 as decimal(38,17))) - promote_precision(cast(engine_size#3 as decimal(38,17)))), DecimalType(38,17), true) as double) <= 0.1)
            :- *(2) Sort [make#1 ASC NULLS FIRST, model#2 ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(make#1, model#2, 5), ENSURE_REQUIREMENTS, [id=#690]
            :     +- *(1) Filter (isnotnull(make#1) AND isnotnull(model#2))
            :        +- *(1) ColumnarToRow
            :           +- FileScan parquet [registration#0,make#1,model#2,engine_size#3] Batched: true, DataFilters: [isno

+------------+------------------+
|registration|   avg(sale_price)|
+------------+------------------+
|     VAXwDvW|2949.7984683595323|
|     oOwbLbj| 2940.165147453083|
|     atj0cou| 2957.128184713376|
|     UPDkPxK| 2957.128184713376|
|     10MC6Fc| 2972.469609724888|
|     JgYTNw4|3000.4284736481864|
|     STRW2oL|2949.7984683595323|
|     EQ1DjHV| 2957.128184713376|
|     J1QgrWA| 2957.128184713376|
|     qFiJQxM| 2972.469609724888|
+------------+------------------+
only showing top 10 rows

CPU times: user 35 ms, sys: 17.3 ms, total: 52.3 ms
Wall time: 1min 32s



#### Spark Tuning Result 

| rank | elapsed | setting | description |
| --- | --- | --- | --- |
| 1 | 41.4 s | custom salting | partition = 3 |
| 2 | 41.6s | adaptive query | AQE enabled |
| 3 | 51.5 s | custom salting | partition = 4 |
| 4 | 1min 32s | default | - |
